# Yelp Restaurant Photos Classification

Yelp held a photo classification competition on Kaggle two years ago. It asked Kagglers
to build a model that automatically tags user-uploaded photos with multiple labels, nine
labels to be exact. In this capstone project, I would be working on designing and
building a Convolutional Neural Network to try to achieve or better the highest
benchmark score. The goal of this project is to assign a set of labels to each photo
correctly. 

## Import Libaries

In [1]:
import os
import math
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import display
from keras.preprocessing import image
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential
from keras import optimizers
from sklearn.datasets import load_files       
from sklearn.model_selection import train_test_split
from PIL import ImageFile
from matplotlib import pyplot as plt


%matplotlib inline

Using TensorFlow backend.


## Import Datasets

**train_photo_to_biz_ids.csv.tgz** contain the mapping
from each photo to its associated business ID. In other words, this mapping allows a
business to have more than one pictures, and we're able to tell which photos belong to
which store.

**train_photos.tgz** contain the actual photos. Photo ID is in each
photo's file name. All the images are user-uploaded, meaning they are non-uniform in
sizes and color images.

**train.csv.tgz** contains each business ID with its associated correct/truth labels. There are
2000 distinct businesses.

In [ ]:
# load train and validation datasets
def load_dataset(path, photos_to_biz, labels, should_split):
    # get filenames
    processed_filenames, processed_labels = [], []
    filenames = os.listdir(path=path)
    for filename in filenames:
        # remove filename that contains underscore
        if '_' not in filename:
            processed_filenames.append(filename)
    
    # delete this line when actually training!!!!!!!!!!!!!!!! only test code works for now
#     processed_filenames = processed_filenames[:8000]
    
    # get each photo's target labels
    for filename in tqdm(processed_filenames):
        bus_id = photos_to_biz.query('train_photo_id=={}'.format(filename.split(".", 1)[0]))['business_id']
        this_labels = labels.query("business_id=={}".format(bus_id.iloc[0]))['labels'].iloc[0]
        
        tmp_labels = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        if isinstance(this_labels, float) and math.isnan(this_labels):
            # empty cell
            pass
        else:
            # multiple labels
            this_labels = this_labels.split(" ")
            for label in this_labels:
                tmp_labels[int(label)] = 1

        processed_labels.append(tmp_labels)
    
    if should_split:
        X_train, X_validation, y_train, y_validation = train_test_split(np.array(processed_filenames), 
                                                                        np.array(processed_labels), 
                                                                        test_size = 0.2, 
                                                                        random_state = 0)

        return X_train, X_validation, y_train, y_validation
    
    # just return filenames and labels
    return np.array(processed_filenames), np.array(processed_labels)

# img_folder = '../'
img_folder = "C:\\Users\\I-Chun Liu\\Documents\\Local_Code\\final_project\\data"

# test_photos_to_biz = pd.read_csv(img_folder + '/test_photo_to_biz.csv.tgz', compression='gzip', sep=',')
# test_photos_to_biz.columns = ['test_photo_id', 'business_id']

train_photo_to_biz_ids = pd.read_csv(img_folder + '/train_photo_to_biz_ids.csv.tgz', compression='gzip', sep=',')
train_photo_to_biz_ids.columns = ['train_photo_id', 'business_id']

train_labels = pd.read_csv(img_folder + '/train.csv.tgz', compression='gzip', sep=',')
train_labels.columns = ['business_id', 'labels']

X_train, X_validation, y_train, y_validation = load_dataset(img_folder + '/train_photos', train_photo_to_biz_ids, train_labels, True)

 12%|█████████████▉                                                                                                         | 27432/234842 [01:36<13:02, 265.05it/s]

In [ ]:
# show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Validation set has {} samples.".format(X_validation.shape[0]))

## Pre-process Training Data

In [ ]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_filenames):
    list_of_tensors = [path_to_tensor(img_folder + '/train_photos/' + img_filename) for img_filename in tqdm(img_filenames)]
    return np.vstack(list_of_tensors)

In [ ]:
train_tensors = paths_to_tensor(X_train).astype('float32')/255
print("Train tenors size: {}".format(train_tensors.shape))

### Image Augmentations

In [ ]:
f1, subplots_array_before = plt.subplots(4,4, figsize=(20, 20))
f1.suptitle('Before Image Augmentations:', fontsize=35)
for i in range(0, 16):
    row, col = 0, 0
    for i in range(0, 16):
        if col == 4:
            col = 0
            row += 1

        subplots_array_before[row, col].imshow(train_tensors[i], interpolation='nearest', aspect='auto')
        col += 1

    # show the plot
    plt.show()
    break

In [ ]:
# perform image augmentations
datagen_train = image.ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.05,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

datagen_train.fit(train_tensors)

f2, subplots_array_after = plt.subplots(4,4, figsize=(20, 20))
f2.suptitle('After Image Augmentations:', fontsize=35)
for X_batch in datagen_train.flow(train_tensors):
    row, col = 0, 0
    for i in range(0, 16):
        if col == 4:
            col = 0
            row += 1

        subplots_array_after[row, col].imshow(X_batch[i], interpolation='nearest', aspect='auto')
        col += 1

    # show the plot
    plt.show()
    break

In [ ]:
from extract_bottleneck_features import *

train_features = extract_Resnet50(train_tensors * 255)

print("Train features shape: {}".format(train_features.shape))

np.save('./bottleneck_features/Resnet50_train', train_features)

# Pre-process / Feature Extraction of Validation Data

Do the same image pre-processing and bottleneck features extraction for validation set:

In [9]:
validation_tensors = paths_to_tensor(X_validation).astype('float32')/255
print("Validation tenors size: {}".format(validation_tensors.shape))

# image pre-processing
datagen_validation = image.ImageDataGenerator(
                    rotation_range=20,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.05,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')
datagen_validation.fit(validation_tensors)

validation_features = extract_Resnet50(validation_tensors * 255)

print("Validation features shape: {}".format(validation_features.shape))

np.save('./bottleneck_features/Resnet50_validation', validation_features)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1600/1600 [00:02<00:00, 628.25it/s]


Validation tenors size: (1600, 224, 224, 3)
processing tensor...
extracting features using Resnet50...
Validation features shape: (1600, 7, 7, 2048)


# Create / Train the Model 

In [10]:
# Load bottleneck features
train_features = np.load('./bottleneck_features/Resnet50_train.npy')
validation_features = np.load('./bottleneck_features/Resnet50_validation.npy')

print("Train features shape: {}".format(train_features.shape))
print("Validation features shape: {}".format(validation_features.shape))

Train features shape: (6400, 7, 7, 2048)
Validation features shape: (1600, 7, 7, 2048)


In [11]:
epochs = 20
batch_size = 32 

model = Sequential()
model.add(GlobalAveragePooling2D(input_shape=train_features.shape[1:]))
model.add(Dense(9, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.fit(train_features, y_train, epochs=epochs, batch_size=batch_size, validation_data=(validation_features, y_validation), verbose=2, shuffle=True)

Train on 6400 samples, validate on 1600 samples
Epoch 1/20
 - 4s - loss: 10.9633 - acc: 0.1289 - val_loss: 10.2990 - val_acc: 0.0919
Epoch 2/20
 - 3s - loss: 10.4404 - acc: 0.1412 - val_loss: 10.4138 - val_acc: 0.1113
Epoch 3/20
 - 3s - loss: 10.2893 - acc: 0.1542 - val_loss: 10.2021 - val_acc: 0.1844
Epoch 4/20
 - 3s - loss: 10.1655 - acc: 0.1603 - val_loss: 9.9945 - val_acc: 0.1294
Epoch 5/20
 - 3s - loss: 10.1140 - acc: 0.1475 - val_loss: 10.1416 - val_acc: 0.2875
Epoch 6/20
 - 3s - loss: 10.0556 - acc: 0.1559 - val_loss: 10.1759 - val_acc: 0.0494
Epoch 7/20
 - 3s - loss: 10.0248 - acc: 0.1541 - val_loss: 10.2804 - val_acc: 0.1069
Epoch 8/20
 - 3s - loss: 9.9883 - acc: 0.1577 - val_loss: 10.6207 - val_acc: 0.0675
Epoch 9/20
 - 3s - loss: 9.9535 - acc: 0.1580 - val_loss: 10.3900 - val_acc: 0.1106
Epoch 10/20
 - 3s - loss: 9.9354 - acc: 0.1630 - val_loss: 10.2158 - val_acc: 0.1594
Epoch 11/20
 - 3s - loss: 9.9049 - acc: 0.1669 - val_loss: 10.2989 - val_acc: 0.1075
Epoch 12/20
 - 3s - 

In [12]:
model.predict(validation_features)[0]

array([0.05492581, 0.11509427, 0.11095002, 0.17964624, 0.02704421,
       0.08612281, 0.20188424, 0.02082847, 0.20350394], dtype=float32)

# References
1. How to use pretrained model in Keras: https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/


In [13]:
model.predict(validation_features)[190]

array([0.07413185, 0.08587088, 0.02096588, 0.07380328, 0.01188321,
       0.02914063, 0.17979518, 0.00446302, 0.5199461 ], dtype=float32)

In [14]:
model.evaluate(validation_features, y_validation, verbose=2)

[10.379753036499023, 0.4375]